## Homework

In [27]:
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [1]:
import numpy as np
import pandas as pd
import copy

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.base import clone

from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [3]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 292 entries, id to price
dtypes: datetime64[ns](1), float64(119), int64(157), object(15)
memory usage: 44.6+ MB


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [5]:
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [6]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
num_columns_train = copy.deepcopy(num_columns)

In [7]:
num_columns_train.remove("price")

Для baseline модели не будем учитывать категориальные признаки. Вместо nan вставим медианное значение признака с помощью imputer

In [8]:
X_train_num = X_train.drop(drop_columns + cat_columns, axis=1)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer.fit_transform(X_train_num)

dt_reg = DecisionTreeRegressor(random_state=42).fit(X_train_num, y_train)

Проверьте качество на отложенной выборке.

In [9]:
X_test_num = X_test.drop(drop_columns + cat_columns, axis=1)
X_test_num = imputer.transform(X_test_num)

y_pred = dt_reg.predict(X_test_num)

rmse_baseline = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE of baseline solution = {rmse_baseline:.1f}")

RMSE of baseline solution = 4355980.7


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [10]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)
num_columns_train.extend(["month_year_cnt", "week_year_cnt"])

/home/shamil/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

У некоторых выборок пропущены некоторые признаки. Воспользуем imputer и вставим вместо пропущенных признаков медианное значение среди всех выборок

In [11]:
median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')

In [12]:
df['max_floor'] = clone(median_imputer).fit_transform(df['max_floor'].to_numpy().reshape(-1,1))
df['floor'] = clone(median_imputer).fit_transform(df['floor'].to_numpy().reshape(-1,1))
df['kitch_sq'] = clone(median_imputer).fit_transform(df['kitch_sq'].to_numpy().reshape(-1,1))

In [13]:
df['month'] = df['timestamp'].dt.month
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['rel_floor'] =  df['floor']/df['max_floor']
df['rel_floor'] = df['rel_floor'].apply(lambda x: 0 if x==np.inf else x)

df['rel_kitch_sq'] = df['kitch_sq']/df['full_sq']
num_columns_train.extend(["month", "dayofweek", "rel_floor", "rel_kitch_sq"])

Для алгоритмов, которые по умолчанию не умеют работать с  категориальными данными сделаем **OneHotEncoding**

In [14]:
df.drop(drop_columns, axis=1, inplace=True)
print(f"Количество признаков в изначальной выборке = {df.shape[1]}")
df_1hot = pd.get_dummies(df, columns=cat_columns)
print(f"Количество признаков в новой выборке = {df_1hot.shape[1]}")

Количество признаков в изначальной выборке = 296
Количество признаков в новой выборке = 1071


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [15]:
X = df.drop('price', axis=1)
y = df['price']
X1hot = df_1hot.drop('price', axis=1)
y1hot = df_1hot['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X1hot_train, X1hot_test, y1hot_train, y1hot_test = train_test_split(X1hot, y1hot, test_size=0.2, shuffle=False)

In [16]:
new_cat_columns = list(set(df_1hot.columns).difference(set(num_columns_train + ['price'])))

После разделения заменим NaN медианном значением признака 

In [17]:
X_train_num = X_train.drop(cat_columns, axis=1)
X_test_num = X_test.drop(cat_columns, axis=1)

imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer_num.fit_transform(X_train_num)
X_test_num = imputer_num.transform(X_test_num)

imputer_1hot = SimpleImputer(missing_values=np.nan, strategy='median')
X1hot_train = imputer_1hot.fit_transform(X1hot_train)
X1hot_test = imputer_1hot.transform(X1hot_test)

Так как у нас стало много признаков, попробуем уменшить размерность использую алгортим основанный на деревьях и сравним предсказания моделей, обученных всех или уменьшенных признаках

In [18]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel

In [19]:
rg = ExtraTreesRegressor(n_estimators=50, random_state=42)
rg.fit(X1hot_train, y1hot_train)

model = SelectFromModel(rg, prefit=True)
X1hot_train_red = model.transform(X1hot_train)
X1hot_test_red = model.transform(X1hot_test)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

Для начала попробуем избавиться от всех категориальных признаков и обучить модели **DecusionTreeRegressor** и **RandomForestRegressor**, которые по умолчанию не умеют работать с категормльными признаками, только на численных признаках

Обучим модель с дефольтными параметрами и посмотрим на параметры

In [21]:
dt_num = DecisionTreeRegressor(random_state=42)
dt_num.fit(X_train_num, y_train)
print(f"Depth of the tree trained on only numerical features: {dt_num.get_depth()}")
print(f"Number of leaves of the tree trained on only numerical features: {dt_num.get_n_leaves()}")
print("---------------------------------------")

dt_1hot = DecisionTreeRegressor(random_state=42)
dt_1hot.fit(X1hot_train, y1hot_train)
print(f"Depth of the tree trained on numerical and categorical features: {dt_1hot.get_depth()}")
print(f"Number of leaves of the tree trained on numerical and categorical features: {dt_1hot.get_n_leaves()}")
print("---------------------------------------")

dt_1hot_red = DecisionTreeRegressor(random_state=42)
dt_1hot_red.fit(X1hot_train_red, y1hot_train)
print(f"Depth of the tree trained on reduced numerical and categorical features: {dt_1hot_red.get_depth()}")
print(f"Number of leaves of the tree reducedtrained on numerical and categorical features: {dt_1hot_red.get_n_leaves()}")

rf_1hot = RandomForestRegressor(random_state=42, n_jobs=3)
rf_1hot.fit(X1hot_train, y1hot_train)

cb_1hot = CatBoostRegressor(random_seed=42, thread_count=3, verbose=0)
cb_1hot.fit(X1hot_train, y1hot_train);

Depth of the tree trained on only numerical features: 48
Number of leaves of the tree trained on only numerical features: 14250
---------------------------------------
Depth of the tree trained on numerical and categorical features: 45
Number of leaves of the tree trained on numerical and categorical features: 14226
---------------------------------------
Depth of the tree trained on reduced numerical and categorical features: 47
Number of leaves of the tree reducedtrained on numerical and categorical features: 14248


In [22]:
y_pred_dt_num = dt_num.predict(X_test_num)
y_pred_dt_1hot = dt_1hot.predict(X1hot_test)
y_pred_dt_1hot_red = dt_1hot_red.predict(X1hot_test_red)
y_pred_rf_1hot = rf_1hot.predict(X1hot_test)
y_pred_cb_1hot = cb_1hot.predict(X1hot_test)

rmse_dt_num = mean_squared_error(y_test, y_pred_dt_num, squared=False)
rmse_dt_1hot = mean_squared_error(y_test, y_pred_dt_1hot, squared=False)
rmse_dt_1hot_red = mean_squared_error(y_test, y_pred_dt_1hot_red, squared=False)
rmse_rf_1hot = mean_squared_error(y_test, y_pred_rf_1hot, squared=False)
rmse_cb_1hot = mean_squared_error(y_test, y_pred_cb_1hot, squared=False)
print(f"RMSE of the prediction on the test set of the tree trained on only numerical features = {rmse_dt_num:.1f}")
print(f"RMSE of the prediction on the test set of the tree trained on both numerical and categorical features = {rmse_dt_1hot:.1f}")
print(f"RMSE of the prediction on the test set of the tree trained on reduced numerical and categorical features = {rmse_dt_1hot_red:.1f}")
print(f"RMSE of the prediction on the test set of the random forest = {rmse_dt_1hot:.1f}")
print(f"RMSE of the prediction on the test set of the catboost model = {rmse_cb_1hot:.1f}")

RMSE of the prediction on the test set of the tree trained on only numerical features = 3722219.0
RMSE of the prediction on the test set of the tree trained on both numerical and categorical features = 3780887.3
RMSE of the prediction on the test set of the tree trained on reduced numerical and categorical features = 3856839.7
RMSE of the prediction on the test set of the random forest = 3780887.3
RMSE of the prediction on the test set of the catboost model = 2588089.7


Добавление категориальных признаков через onehotencoding не улучшил метрику! А уменьшение размерности даже сильно ухудшил метрику. Скорее всего произошло переобучение. Подберем гиперпараметры модели. Начнем с дерева.

In [142]:
dt = DecisionTreeRegressor(random_state=42)
parameters_dt = {"max_depth":[3, 5, 10, 13, 15],
                "max_features":["auto", "sqrt", "log2"],
                "min_samples_leaf":[1, 5, 10, 15],
                "min_samples_split":[2, 4, 6]}

dt_param_search = GridSearchCV(dt, parameters_dt, scoring='neg_mean_squared_error', n_jobs=-1)
dt_param_search.fit(X1hot_train_red, y1hot_train);

In [143]:
dt_param_search.best_params_

{'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 15,
 'min_samples_split': 2}

Перейдем к слуайному лесу. Тут можно было бы больше параметров перебрать, но слишком уж долго считается. Здесь я не рассматриваю глубину дерева, потому что случайный робастен по отношению к глубине леса

In [24]:
rf = RandomForestRegressor(random_state=42)
parameters_rf = {"n_estimators":[100,200,300],
                "max_samples":[2/3,1/3],
                "max_depth":[6,10,15],
                "warm_start":[True]}

rf_param_search = GridSearchCV(rf, parameters_rf, scoring='neg_mean_squared_error', n_jobs=-1)
rf_param_search.fit(X1hot_train_red, y1hot_train)

GridSearchCV(estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [6, 10, 15],
                         'max_samples': [0.6666666666666666,
                                         0.3333333333333333],
                         'min_samples_leaf': [2, 5, 10],
                         'n_estimators': [100, 200, 300],
                         'warm_start': [True]},
             scoring='neg_mean_squared_error')

In [25]:
rf_param_search.best_params_

{'max_depth': 15,
 'max_samples': 0.6666666666666666,
 'min_samples_leaf': 2,
 'n_estimators': 200,
 'warm_start': True}

В случае catboost тоже можно было поискать оптимальные гипермараметры в большем множестве, но время поиска тогда растянется на дни

In [20]:
parameters_cb = {"iterations":[250,500,1000,2000],
                "depth":[5,7,9],
                "l2_leaf_reg":[3,5,10]}

cb = CatBoostRegressor(verbose=0, random_seed=42)

grid_search_cb = GridSearchCV(cb, parameters_cb, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_cb.fit(X1hot_train_red, y1hot_train)

GridSearchCV(estimator=<catboost.core.CatBoostRegressor object at 0x7f91b54748d0>,
             n_jobs=-1,
             param_grid={'depth': [5, 7, 9],
                         'iterations': [250, 500, 1000, 2000],
                         'l2_leaf_reg': [3, 5, 10]},
             scoring='neg_mean_squared_error')

In [21]:
grid_search_cb.best_params_

{'depth': 7, 'iterations': 2000, 'l2_leaf_reg': 10}

Обучим лучшие модели на всей выборке

In [22]:
dt_best = DecisionTreeRegressor(max_depth=10, max_features='auto', 
                                splitter='best', min_samples_leaf=15, 
                                min_samples_split=2, random_state=42)
dt_best.fit(X1hot_train_red, y1hot_train)

rf_best = RandomForestRegressor(n_estimators=200, max_samples=2/3, 
                                min_samples_leaf=2, max_depth=15, random_state=42)
rf_best.fit(X1hot_train_red, y1hot_train)

cb_best = CatBoostRegressor(iterations=2000, depth=7, l2_leaf_reg=10)
cb_best.fit(X1hot_train_red, y1hot_train, verbose=0)

Сравним лучшие модели по метрике

In [23]:
y_pred_dt = dt_best.predict(X1hot_test_red)
y_pred_rf = rf_best.predict(X1hot_test_red)
y_pred_cb = cb_best.predict(X1hot_test_red)

print(f"RMSE decision tree: {mean_squared_error(y_test, y_pred_dt, squared=False):.2f}")
print(f"RMSE random forest: {mean_squared_error(y_test, y_pred_rf, squared=False):.2f}")
print(f"RMSE boosting: {mean_squared_error(y_test, y_pred_cb, squared=False):.2f}")

RMSE decision tree: 3119856.37
RMSE random forest: 2737697.43
RMSE boosting: 2566334.15


Победитель: catboost

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

Разделим данные по признаку product_type

In [24]:
invest_df = df_1hot[df_1hot['product_type_Investment']==1]
owner_df = df_1hot[df_1hot['product_type_OwnerOccupier']==1]

invest_df = invest_df.drop(['product_type_Investment', 'product_type_OwnerOccupier'], axis=1)
owner_df = owner_df.drop(['product_type_Investment', 'product_type_OwnerOccupier'], axis=1);

In [25]:
Xinvest = invest_df.drop('price', axis=1)
yinvest = invest_df['price']

Xowner = owner_df.drop('price', axis=1)
yowner = owner_df['price']

Разделим на тренировочную и тестовые выборки

In [26]:
Xinvest_train, Xinvest_test, yinvest_train, yinvest_test = train_test_split(Xinvest, yinvest, test_size=0.2, shuffle=False)
Xowner_train, Xowner_test, yowner_train, yowner_test = train_test_split(Xowner, yowner, test_size=0.2, shuffle=False)

Заменим nan на медианное значение признака

In [27]:
invest_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
owner_imputer = SimpleImputer(missing_values=np.nan, strategy='median')

Xinvest_train = invest_imputer.fit_transform(Xinvest_train)
Xinvest_test = invest_imputer.transform(Xinvest_test)

Xowner_train = owner_imputer.fit_transform(Xowner_train)
Xowner_test = owner_imputer.transform(Xowner_test)

Уменьшим размерность данных (количество признаков)

In [28]:
etr_invest = ExtraTreesRegressor(n_estimators=50, random_state=42)
etr_invest.fit(Xinvest_train, yinvest_train)

selector_invest = SelectFromModel(etr_invest, prefit=True)
Xinvest_train_red = selector_invest.transform(Xinvest_train)
Xinvest_test_red = selector_invest.transform(Xinvest_test)

etr_owner = ExtraTreesRegressor(n_estimators=50, random_state=42)
etr_owner.fit(Xowner_train, yowner_train)

selector_owner = SelectFromModel(etr_invest, prefit=True)
Xowner_train_red = selector_owner.transform(Xowner_train)
Xowner_test_red = selector_owner.transform(Xowner_test)

Обучим лучшую модель на новых данных

In [29]:
model_invest = CatBoostRegressor(iterations=2000, depth=7, l2_leaf_reg=10, verbose=0)
model_invest.fit(Xinvest_train_red, yinvest_train)

model_owner = CatBoostRegressor(iterations=2000, depth=7, l2_leaf_reg=10, verbose=0)
model_owner.fit(Xowner_train_red, yowner_train)

yinvest_pred = model_invest.predict(Xinvest_test_red)
yowner_pred = model_owner.predict(Xowner_test_red)

print(f"RMSE Investment = {mean_squared_error(yinvest_test, yinvest_pred, squared=False):.2f}")
print(f"RMSE Owner = {mean_squared_error(yowner_test, yowner_pred, squared=False):.2f}")

RMSE Investment = 2905520.65
RMSE Owner = 1939059.53


Модель намного лучше предсказывает target для квартир, которые продаются для жилья.

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [31]:
Xinvest_train2, Xinvest_val, yinvest_train2, yinvest_val = train_test_split(Xinvest_train_red, yinvest_train, test_size=0.2, shuffle=True)

In [32]:
model_invest_cb = CatBoostRegressor(iterations=2000, depth=7, l2_leaf_reg=10, verbose=0)
model_invest_cb.fit(Xinvest_train2, yinvest_train2)

model_invest_huber = HuberRegressor(max_iter=2e+3)
model_invest_huber.fit(Xinvest_train2, yinvest_train2)

HuberRegressor(max_iter=2000.0)

Создадим сетку весов

In [41]:
w1 = np.linspace(0, 1, 15)
w = zip(w1, 1-w1)

In [42]:
rmse_invest = []
for w1, w2 in w:
    y_pred1 = model_invest_cb.predict(Xinvest_val)
    y_pred2 = model_invest_huber.predict(Xinvest_val)
    y_pred = w1*y_pred1 + w2*y_pred2
    rmse_invest.append(tuple((w1, w2, mean_squared_error(yinvest_val, y_pred, squared=False))))

best_w1, best_w2, _ = min(rmse_invest, key=lambda x: x[2])
print(f"Best w1 = {best_w1:.4f}, w2 = {best_w2:.4f}")

Best w1 = 0.9286, w2 = 0.0714


In [43]:
yinvest_pred = best_w1*model_invest_cb.predict(Xinvest_test_red) + \
                best_w2*model_invest_huber.predict(Xinvest_test_red)

print(f"RMSE Investment = {mean_squared_error(yinvest_test, yinvest_pred, squared=False):.2f}")

RMSE Investment = 2857284.51


Более сложная модель: комбинация `CatboostRegressor` и `HuberRegressor` смогла незначительно улучшить метрику на тестовой выборке